In [46]:
import pandas as pd
import numpy as np
import tensorflow as tf
#tf.debugging.set_log_device_placement(True)
#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(len(gpus))

In [47]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [48]:
#data load, F.txt 파일이 있는 절대 경로를 입력하시면 됩니다.
#data 확인을 위하여 출력해봅니다. column name에 데이터가 들어가있어서 처리해주어야 합니다.
# N = 5
data = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/F.txt', sep = '\t')
data

,-0.07554167,0.15564174,0.17413366,0.13918874,0.14987264,-0.02049801,0.09800520,-0.06676333,-0.08714528,0.08003181,...,0.17935742,-0.05583790,-0.06579545,0.06183800,-0.20620972,0.05911842,-0.05977414,-0.02171137,0.10180243,-0.14521111
0,-0.005535,0.006658,-0.000676,0.001881,-0.008176,0.000366,-0.002526,-0.003462,0.008640,-0.013094,...,-0.004038,0.006765,-0.004190,-0.006789,0.009556,0.002606,0.005746,-0.005756,-0.001540,-0.007515
1,0.000074,-0.000047,0.000003,-0.000040,-0.000058,0.000076,-0.000033,-0.000008,0.000031,-0.000016,...,0.000021,0.000052,0.000009,-0.000076,0.000007,-0.000007,-0.000038,-0.000061,-0.000087,0.000012


In [49]:
arr = []
for i in range(len(data.iloc[0,:])):
    arr.append(i+1)

In [50]:
#load data
#column name에 data가 들어가있기 때문에 "names = arr" 로 처리해 줍니다. arr은 1 ~ len(F)의 숫자가 담겨있습니다.
dataF = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/F.txt', sep = '\t', names = arr)
dataQ = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/Q.txt', sep = '\t', names = arr)
data_nF = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/nF.txt', sep = '\t', names = arr)

In [51]:
def make_arr(A):
    arA = []
    for j in range(len(A.iloc[0, :])):
        arA1 = []
        for i in range(len(A.iloc[:,0])):
            tmpA = A.iloc[:,j][i]
            arA1.append(tmpA)
        arA.append(arA1)
    return arA

In [52]:
#make array
arrF = make_arr(dataF)
dataF = np.array(arrF)

arrQ = make_arr(dataQ)
dataQ = np.array(arrQ)

arr_nF = make_arr(data_nF)
data_nF = np.array(arr_nF)

In [53]:
# N = 5
A = []
for n in range(1,6):
    x = np.exp(-(n**2))
    A.append(x)
A = np.diag(A)


In [54]:
# data 분할

import math
train_size = math.floor(len(dataF)*0.6) # train : 60%
val_size = math.floor(len(dataF)*0.2) #val : 20%
test_size = math.floor(len(dataF)*0.2) #test : 20%
#generate F_data, F_val, F_test
F_data = dataF[:train_size, :]
F_val = dataF[train_size:(val_size + train_size), :]
F_test = dataF[(val_size + train_size):(val_size + train_size + test_size), :]

#generate Q_data, Q_val, Q_test
Q_data = dataQ[:train_size, :]
Q_val = dataQ[train_size:(val_size + train_size), :]
Q_test = dataQ[(val_size + train_size):(val_size + train_size + test_size), :]

#generate nF_data, nF_val, nF_test
nF_data = data_nF[:train_size, :]
nF_val = data_nF[train_size:(val_size + train_size), :]
nF_test = data_nF[(val_size + train_size):(val_size + train_size + test_size), :]

print(f'{F_data.shape}, {F_test.shape}, {F_val.shape}')
print(f'{Q_data.shape}, {Q_test.shape}, {Q_val.shape}')
print(f'{nF_data.shape}, {nF_test.shape}, {nF_val.shape}')

(60000, 3), (20000, 3), (20000, 3)
(60000, 3), (20000, 3), (20000, 3)
(60000, 3), (20000, 3), (20000, 3)


In [55]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential() #Sequentioal

model.add(keras.layers.Dense(input_dim = 3, units = 10, activation= "tanh",kernel_initializer ="HeNormal" )) 
model.add(keras.layers.Dense(20, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))  
model.add(keras.layers.Dense(100, activation= "tanh")) 
model.add(keras.layers.Dense(120, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))
model.add(keras.layers.Dense(3, activation= "tanh"))

In [56]:
with tf.device('/GPU:0'):
    F_train = tf.constant(F_data)
    Q_train = tf.constant(Q_data)
    nF_train = tf.constant(nF_data)

In [57]:
model.compile(loss = "mse",
              optimizer = "ADAM",
              metrics = ["accuracy"])
history = model.fit(x = F_train, y = Q_train, validation_data=(F_val, Q_val),epochs = 50)

Epoch 1/50
1875/1875 [==============================] - 1s 514us/step - loss: 0.0438 - accuracy: 0.7424 - val_loss: 0.0434 - val_accuracy: 0.7491
Epoch 2/50
1875/1875 [==============================] - 1s 495us/step - loss: 0.0426 - accuracy: 0.7463 - val_loss: 0.0432 - val_accuracy: 0.7471
Epoch 3/50
1875/1875 [==============================] - 1s 491us/step - loss: 0.0425 - accuracy: 0.7482 - val_loss: 0.0432 - val_accuracy: 0.7510
Epoch 4/50
1875/1875 [==============================] - 1s 486us/step - loss: 0.0425 - accuracy: 0.7486 - val_loss: 0.0431 - val_accuracy: 0.7501
Epoch 5/50
1875/1875 [==============================] - 1s 493us/step - loss: 0.0424 - accuracy: 0.7520 - val_loss: 0.0426 - val_accuracy: 0.7553
Epoch 6/50
1875/1875 [==============================] - 1s 489us/step - loss: 0.0221 - accuracy: 0.8559 - val_loss: 0.0025 - val_accuracy: 0.9603
Epoch 7/50
1875/1875 [==============================] - 1s 489us/step - loss: 0.0015 - accuracy: 0.9672 - val_loss: 7.1097e-

In [58]:
#F 학습 후 F accuracy
result = model.evaluate(F_test, Q_test)

625/625 [==============================] - 0s 261us/step - loss: 1.2018e-04 - accuracy: 0.9918


In [59]:
#F 학습 후 nF accuracy
result = model.evaluate(nF_test, Q_test)

625/625 [==============================] - 0s 267us/step - loss: 0.3131 - accuracy: 0.5096


In [60]:
model.compile(loss = "mse",
              optimizer = "adam",
              metrics = ["accuracy"])
history = model.fit(x = nF_train, y = Q_train, validation_data=(nF_val, Q_val),epochs = 50)

Epoch 1/50
1875/1875 [==============================] - 1s 531us/step - loss: 0.0484 - accuracy: 0.7368 - val_loss: 0.0449 - val_accuracy: 0.7385
Epoch 2/50
1875/1875 [==============================] - 1s 500us/step - loss: 0.0430 - accuracy: 0.7452 - val_loss: 0.0439 - val_accuracy: 0.7350
Epoch 3/50
1875/1875 [==============================] - 1s 498us/step - loss: 0.0427 - accuracy: 0.7502 - val_loss: 0.0425 - val_accuracy: 0.7564
Epoch 4/50
1875/1875 [==============================] - 1s 501us/step - loss: 0.0418 - accuracy: 0.7526 - val_loss: 0.0417 - val_accuracy: 0.7606
Epoch 5/50
1875/1875 [==============================] - 1s 514us/step - loss: 0.0412 - accuracy: 0.7557 - val_loss: 0.0418 - val_accuracy: 0.7513
Epoch 6/50
1875/1875 [==============================] - 1s 500us/step - loss: 0.0410 - accuracy: 0.7591 - val_loss: 0.0413 - val_accuracy: 0.7560
Epoch 7/50
1875/1875 [==============================] - 1s 496us/step - loss: 0.0406 - accuracy: 0.7605 - val_loss: 0.0407 -

In [61]:
#nF 학습 후 nF accuracy
result = model.evaluate(nF_test, Q_test)

625/625 [==============================] - 0s 263us/step - loss: 0.0400 - accuracy: 0.7581
